# Clustering Crypto

In [1]:
# Install the altair library for plotting
!pip install -U altair

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 KB 45.6 MB/s eta 0:00:00
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [3]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
import altair as alt
from path import Path
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Fetching Cryptocurrency Data

In [ ]:
# Use the following endpoint to fetch json data
#url = "https://min-api.cryptocompare.com/data/all/coinlist"

In [ ]:
# Create a DataFrame 
# HINT: You will need to use the 'Data' key from the json response, then transpose the DataFrame.

# # Get crypto data using call to CryptoCompare API endpoint
# crypto_response_data = requests.get(url)

# # Convert response from api call to json format
# crypto_response_data_json = crypto_response_data.json()

# # Reach into json response content and select the Data then Create DataFrame 
# crypto_data = pd.DataFrame(crypto_response_data_json["Data"])

# # Transponse Date Frame and inspect
# crypto_df = crypto_data.T
# crypto_df.head()


In [5]:
# Alternatively, use the provided csv file:
file_path = Path("../Resources/crypto_data.csv")

# Create a DataFrame
crypto_df = pd.read_csv(file_path, delimiter=",", index_col=0)

### Data Preprocessing

In [6]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
crypto_df.head(20)


,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
2015,2015 coin,X11,True,PoW/PoS,NaN,0
BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


In [7]:
# Keep only cryptocurrencies that are trading
crypto_df = crypto_df.loc[crypto_df['IsTrading'] == True]

In [8]:
# Keep only cryptocurrencies with a working algorithm
crypto_df = crypto_df.loc[crypto_df['Algorithm'] != 'N/A']

In [9]:
# Remove the "IsTrading" column
crypto_df.drop(columns = 'IsTrading', inplace=True)

In [10]:
# Remove rows with at least 1 null value
crypto_df.dropna(inplace = True)

In [11]:
# Remove rows with cryptocurrencies having no coins mined
crypto_df = crypto_df.loc[crypto_df['TotalCoinsMined'] > 0]         #Select only cryptos that have mined coins > 0

In [12]:
# Drop rows where there are 'N/A' text values
na_columns = crypto_df.columns[crypto_df.isin(['N/A']).any()]     #Identify all columns that have N/A

for i in na_columns:
    crypto_df = crypto_df.loc[crypto_df[i] != 'N/A']              #Drop rows that have n/a in the selected column

In [13]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
coins_name = pd.DataFrame(crypto_df['CoinName'])
coins_name.set_index(crypto_df.index)
coins_name.head(10)

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum
LTC,Litecoin
DASH,Dash
XMR,Monero
ETC,Ethereum Classic
ZEC,ZCash


In [14]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
crypto_df.drop(columns="CoinName", inplace=True)
crypto_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0


In [15]:
crypto_df.dtypes   #checking the data type of column to see if any need to be converted

Algorithm           object
ProofType           object
TotalCoinsMined    float64
TotalCoinSupply     object
dtype: object

In [16]:
crypto_df['TotalCoinSupply'] = crypto_df['TotalCoinSupply'].astype(float)   #Convert TotalCoinSupply to float

In [17]:
# Create dummy variables for text features
X = pd.get_dummies(crypto_df, columns=['Algorithm', 'ProofType'])

In [18]:
# Standardize data
X_scaled = StandardScaler().fit_transform(X)
print(X_scaled[0:5])

[[-0.11710817 -0.1528703  -0.0433963  -0.0433963  -0.0433963  -0.06142951
  -0.07530656 -0.0433963  -0.06142951 -0.06142951 -0.0433963  -0.0433963
  -0.19245009 -0.06142951 -0.09740465 -0.0433963  -0.11547005 -0.07530656
  -0.0433963  -0.0433963  -0.15191091 -0.0433963  -0.13118084 -0.0433963
  -0.0433963  -0.08703883 -0.0433963  -0.0433963  -0.0433963  -0.0433963
  -0.06142951 -0.0433963  -0.08703883 -0.08703883 -0.08703883 -0.0433963
  -0.13118084 -0.13840913 -0.13840913 -0.0433963  -0.06142951 -0.0433963
  -0.07530656 -0.18168574 -0.0433963  -0.0433963  -0.0433963  -0.07530656
  -0.15826614 -0.31491833 -0.0433963  -0.08703883 -0.07530656 -0.06142951
   1.38675049 -0.0433963  -0.0433963  -0.06142951 -0.0433963  -0.0433963
  -0.0433963  -0.0433963  -0.0433963  -0.0433963  -0.0433963  -0.0433963
  -0.39879994 -0.0433963  -0.18168574 -0.0433963  -0.08703883 -0.08703883
  -0.10680283 -0.0433963  -0.13118084 -0.0433963  -0.0433963  -0.0433963
  -0.0433963  -0.07530656 -0.43911856 -0.04339

### Reducing Dimensions Using PCA

In [19]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(X_scaled)

In [20]:
# Create a DataFrame with the principal components data
pcs_df = pd.DataFrame(
    data=crypto_pca, columns=["PC 1", "PC 2", "PC 3"], index=crypto_df.index)

pcs_df.head(10)

,PC 1,PC 2,PC 3
42,-0.335373,1.034470,-0.530703
404,-0.318689,1.034897,-0.530963
1337,2.310268,1.712731,-0.649224
BTC,-0.145528,-1.305646,0.141739
ETH,-0.130083,-2.063055,0.401138
LTC,-0.166345,-1.116628,0.008180
DASH,-0.393790,1.222860,-0.517092
XMR,-0.172400,-2.223373,0.341634
ETC,-0.128523,-2.063121,0.401126
ZEC,-0.139972,-2.144794,0.441133


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [21]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)


# Create the Elbow Curve using altair
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
alt.Chart(df_elbow, title="Elbow Curve").mark_circle(size=60).encode(
    x="k", 
    y="inertia", 
    color=alt.Color('inertia', scale=alt.Scale(scheme='viridis')),
    tooltip=['inertia', 'k']
)

C:\ProgramData\Anaconda3\envs\pyvizenv\lib\site-packages\sklearn\cluster\_kmeans.py:1037: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  "KMeans is known to have a memory leak on Windows "


alt.Chart(...)

Running K-Means with `k=4`

In [22]:
# Initialize the K-Means model
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)

# Create a new DataFrame including predicted clusters and cryptocurrencies features
clustered_df = pd.DataFrame({
    "Algorithm": crypto_df.Algorithm,
    "ProofType": crypto_df.ProofType,
    "TotalCoinsMined": crypto_df.TotalCoinsMined,
    "TotalCoinSupply": crypto_df.TotalCoinSupply,                     
    "PC 1": pcs_df['PC 1'],
    "PC 2": pcs_df['PC 2'],
    "PC 3": pcs_df['PC 3'],
    "CoinName": coins_name.CoinName,
    "Class": model.labels_,
    }, index=crypto_df.index
)
clustered_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,4.200000e+01,-0.335373,1.034470,-0.530703,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,5.320000e+08,-0.318689,1.034897,-0.530963,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,3.141593e+11,2.310268,1.712731,-0.649224,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,2.100000e+07,-0.145528,-1.305646,0.141739,Bitcoin,3
ETH,Ethash,PoW,1.076842e+08,0.000000e+00,-0.130083,-2.063055,0.401138,Ethereum,3


### Visualizing Results

#### Scatter Plot with Tradable Cryptocurrencies

In [23]:
# Scale data to create the scatter plot
clustered_scaled_df = pd.DataFrame(StandardScaler().fit_transform(clustered_df[["TotalCoinsMined", "TotalCoinSupply"]]), index=clustered_df.index)
clustered_scaled_df.head()

,0,1
42,-0.117108,-0.152870
404,-0.093970,-0.145009
1337,0.524946,4.489424
BTC,-0.116715,-0.152560
ETH,-0.114747,-0.152870


In [24]:
# Plot the scatter with x="PC 1" and y="PC 2"
alt.Chart(clustered_df, title="Scatter Plot- PC 1 vs PC 2", width=800, height=400).mark_point(size=60).encode(
    x="PC 1", 
    y="PC 2", 
    color=alt.Color('Class', scale=alt.Scale(scheme='viridis')),
    # Add the required tool tips "CoinName", "Algorithm", "TotalCoinsMined", "TotalCoinSupply"
    tooltip=["CoinName", "Algorithm", "TotalCoinsMined", "TotalCoinSupply"]
    
)

alt.Chart(...)

In [25]:
#Use the altair scatter plot to visualize the tradable cryptocurrencies, x="TotalCoinsMined" and y="TotalCoinSupply" for the axes
alt.Chart(clustered_df, title="Scatter Plot- TotalCoinsMined vs TotalCoinSupply", width=800, height=200).mark_point(size=60).encode(
    x="TotalCoinsMined", 
    y="TotalCoinSupply", 
    color=alt.Color('Class', scale=alt.Scale(scheme='viridis')),
    # Add the required tool tips "CoinName", "Algorithm", "TotalCoinsMined", "TotalCoinSupply"
    tooltip=["CoinName", "Algorithm", "TotalCoinsMined", "TotalCoinSupply"]
    
)

alt.Chart(...)

#### Table of Tradable Cryptocurrencies

In [26]:
# Print the total number of tradable cryptocurrencies using the display() command
display(clustered_df[['CoinName','Algorithm', 'ProofType', 'TotalCoinSupply', 'TotalCoinsMined', 'Class']])

,CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class
42,42 Coin,Scrypt,PoW/PoS,4.200000e+01,4.199995e+01,0
404,404Coin,Scrypt,PoW/PoS,5.320000e+08,1.055185e+09,0
1337,EliteCoin,X13,PoW/PoS,3.141593e+11,2.927942e+10,0
BTC,Bitcoin,SHA-256,PoW,2.100000e+07,1.792718e+07,3
ETH,Ethereum,Ethash,PoW,0.000000e+00,1.076842e+08,3
...,...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2.000000e+09,0
GAP,Gapcoin,Scrypt,PoW/PoS,2.500000e+08,1.493105e+07,0
BDX,Beldex,CryptoNight,PoW,1.400223e+09,9.802226e+08,3
ZEN,Horizen,Equihash,PoW,2.100000e+07,7.296538e+06,3
